In [1]:
from openff.qcsubmit.common_structures import QCSpec, SCFProperties
from openff.qcsubmit.factories import OptimizationDatasetFactory
from openff.qcsubmit import workflow_components
import numpy as np
from qcelemental.models.results import WavefunctionProtocolEnum
from qcportal.models.common_models import DriverEnum
from openeye import oechem

In [2]:
factory = OptimizationDatasetFactory(qc_specifications={'default': QCSpec(method='b3lyp-d3bj', 
                                                                   basis='dzvp', 
                                                                   program='psi4', 
                                                                   spec_name='default', 
                                                                   spec_description='Standard OpenFF optimization quantum chemistry specification.',
                                                                   implicit_solvent=None, 
                                                                   maxiter=200, 
                                                                   scf_properties=[SCFProperties.Dipole, SCFProperties.Quadrupole, SCFProperties.WibergLowdinIndices, 
                                                                                   SCFProperties.MayerIndices, SCFProperties.MBISCharges],
                                                                         )},
                                    )
factory.add_workflow_components(workflow_components.StandardConformerGenerator(max_conformers=10))
factory.dict()

{'qc_specifications': {'default': {'method': 'b3lyp-d3bj',
   'basis': 'dzvp',
   'program': 'psi4',
   'spec_name': 'default',
   'spec_description': 'Standard OpenFF optimization quantum chemistry specification.',
   'store_wavefunction': 'none',
   'implicit_solvent': None,
   'maxiter': 200,
   'scf_properties': ['dipole',
    'quadrupole',
    'wiberg_lowdin_indices',
    'mayer_indices',
    'mbis_charges'],
   'keywords': None}},
 'driver': 'gradient',
 'priority': 'normal',
 'dataset_tags': ['openff'],
 'compute_tag': 'openff',
 'type': 'OptimizationDatasetFactory',
 'workflow': [{'type': 'StandardConformerGenerator',
   'rms_cutoff': None,
   'max_conformers': 10,
   'clear_existing': True}],
 'optimization_program': {'program': 'geometric',
  'coordsys': 'dlc',
  'enforce': 0.0,
  'epsilon': 1e-05,
  'reset': True,
  'qccnv': False,
  'molcnv': False,
  'check': 0,
  'trust': 0.1,
  'tmax': 0.3,
  'maxiter': 300,
  'convergence_set': 'GAU',
  'constraints': {}}}

In [3]:
dataset = factory.create_dataset(dataset_name="OpenFF Gen2 Optimization Dataset Protomers v1.0", molecules="confs_with_different_protonation_states.smi", tagline="Reasonable protomers enumerated", description="Reasonable protomers enumerated for all the molecules in Gen2 optimization datasets and if at least one enumerated reasonable protonation state doesn't match with the molecule's current state then those molecules are added in this dataset. Molecules with nitro groups are excluded from this filtering since pentavalent nitrogens (O=N(-*)=O) are not appropriate.")

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): Reasonable protomer: natoms = 13, net_chg = -1
Problematic atoms are:
Atom atomic num: 16, name: , idx: 7, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 3, aromatic: True, chiral: False
bond order: 2, chiral: False to atom atomic num: 8, name: , idx: 8, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 9, aromatic: False, chiral: False

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): Reasonable protomer: natoms = 29, net_chg = -1
Problematic bonds are: [<openeye.oechem.OEBondBase; proxy of <Swig Object of type 'OEChem::OEBondBase *' at 0x7fbf367eff60> >]

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): Reasonable protomer: natoms = 26, net

Preparation                   :  55%|████▉    | 138/252 [00:04<00:03, 29.38it/s]Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic bonds are: [<openeye.oechem.OEBondBase; proxy of <Swig Object of type 'OEChem::OEBondBase *' at 0x7fbf3617c720> >]

Preparation                   :  85%|███████▋ | 215/252 [00:09<00:03, 10.33it/s]Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 6, name: , idx: 21, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 20, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 22, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 1, name: , idx: 25, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 26, aromatic: F

In [4]:
confs = np.array([len(mol.conformers) for mol in dataset.molecules])

print("Number of unique molecules       ", dataset.n_molecules)
print("Number of filtered molecules     ", dataset.n_filtered)
print("Number of conformers             ", dataset.n_records)
print("Number of conformers min mean max",
      confs.min(), "{:6.2f}".format(confs.mean()), confs.max())

masses = []
for molecule in dataset.molecules:
    oemol = molecule.to_openeye()
    mass = oechem.OECalculateMolecularWeight(oemol)
    masses.append(mass)

print(f'Mean molecular weight: {np.mean(np.array(masses)):.2f}')
print(f'Max molecular weight: {np.max(np.array(masses)):.2f}')
print("Charges:", sorted(set(m.total_charge/m.total_charge.unit for m in dataset.molecules)))

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 15, name: , idx: 7, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 1, aromatic: False, chiral: False
bond order: 2, chiral: False to atom atomic num: 8, name: , idx: 8, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 1, name: , idx: 9, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 10, aromatic: False, chiral: False

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 16, name: , idx: 9, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 4, aromatic: True, chiral: False
bond order: 2, chiral: False to atom atomic num: 8, name: , idx: 10, a

Number of unique molecules        110
Number of filtered molecules      0
Number of conformers              610
Number of conformers min mean max 1   5.55 10


Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic bonds are: [<openeye.oechem.OEBondBase; proxy of <Swig Object of type 'OEChem::OEBondBase *' at 0x7fbf34109720> >]

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 6, name: , idx: 21, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 20, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 22, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 1, name: , idx: 25, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 26, aromatic: False, chiral: False

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
A

Mean molecular weight: 280.17
Max molecular weight: 542.59


Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic bonds are: [<openeye.oechem.OEBondBase; proxy of <Swig Object of type 'OEChem::OEBondBase *' at 0x7fbf341c8930> >]

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 6, name: , idx: 21, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 20, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 22, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 1, name: , idx: 25, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 26, aromatic: False, chiral: False



Charges: [-3.0, -2.0, -1.0, 0.0, 1.0, 2.0]


In [5]:
dataset.visualize("protomers_not_present_in_gen2_optimization_sets.pdf")

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 15, name: , idx: 7, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 1, aromatic: False, chiral: False
bond order: 2, chiral: False to atom atomic num: 8, name: , idx: 8, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 1, name: , idx: 9, aromatic: False, chiral: False
bond order: 1, chiral: False to atom atomic num: 8, name: , idx: 10, aromatic: False, chiral: False

Warning (not error because allow_undefined_stereo=True): OEMol has unspecified stereochemistry. oemol.GetTitle(): 
Problematic atoms are:
Atom atomic num: 16, name: , idx: 9, aromatic: False, chiral: True with bonds:
bond order: 1, chiral: False to atom atomic num: 6, name: , idx: 4, aromatic: True, chiral: False
bond order: 2, chiral: False to atom atomic num: 8, name: , idx: 10, a

In [6]:
dataset.metadata.submitter = 'Pavan Behara'

In [7]:
dataset.metadata.long_description_url = 'https://github.com/openforcefield/qca-dataset-submission/tree/master/submissions/2021-12-21-OpenFF-Gen2-Optimization-Set-Protomers'

In [8]:
dataset.metadata

Metadata(submitter='Pavan Behara', creation_date=datetime.date(2021, 12, 21), collection_type='OptimizationDataset', dataset_name='OpenFF Gen2 Optimization Dataset Protomers v1.0', short_description='Reasonable protomers enumerated', long_description_url=HttpUrl('https://github.com/openforcefield/qca-dataset-submission/tree/master/submissions/2021-12-21-OpenFF-Gen2-Optimization-Set-Protomers', scheme='https', host='github.com', tld='com', host_type='domain', path='/openforcefield/qca-dataset-submission/tree/master/submissions/2021-12-21-OpenFF-Gen2-Optimization-Set-Protomers'), long_description="Reasonable protomers enumerated for all the molecules in Gen2 optimization datasets and if at least one enumerated reasonable protonation state doesn't match with the molecule's current state then those molecules are added in this dataset. Molecules with nitro groups are excluded from this filtering since pentavalent nitrogens (O=N(-*)=O) are not appropriate.", elements={'O', 'F', 'S', 'Br', 'C

In [9]:
dataset.export_dataset("dataset.json.bz2")